###Mounting the google drive

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Importing Libraries

In [4]:
import pandas as pd
import os 

###Bounding Box

In [65]:
class Bounding_Box:

  def unique_identifier(path,csv_location):
    
    #change directory 
    os.chdir(path)

    #unique_identifier for each label (Total 600 labels)
    classes_data = pd.read_csv(csv_location)

    #unique_identifier for person & car class
    print(classes_data[classes_data[' Class'] == 'Person'])
    print(classes_data[classes_data[' Class'] == 'Car'])

    return classes_data

  def annotation_data(csv_location):
    annotation_data = pd.read_c     sv(csv_location, usecols = ['ImageID','LabelName', 'XMin','XMax', 'YMax', 'YMin'])
    #print(annotation_data.head())
    return annotation_data

  def BB_Calculation(annotation_data,classes, class_strings):
    
    for i in range(len(class_strings)):
      annotation_data.loc[annotation_data['LabelName'] == class_strings[i], 'classnumber'] = int(i)

    annotation_data['center x'] = annotation_data['XMax'] + annotation_data['XMin'] / 2
    annotation_data['center y'] = annotation_data['YMax'] + annotation_data['YMin'] / 2

    annotation_data['width'] = annotation_data['XMax'] - annotation_data['XMin']
    annotation_data['height'] = annotation_data['YMax'] - annotation_data['YMin'] 

    YOLO_VALUES = annotation_data.loc[:,['ImageID', 'classnumber', 'center x', 'center y', 'width', 'height']].copy()
    
    return YOLO_VALUES

    



###Yolo Format

In [ ]:
class yolo_format:

  def annotation_file_txt(YOLO_VALUES):

    image_path = '/content/drive/MyDrive/YOLOV4/Dataset_And_Annotations/Dataset/train/Person_Car'
    os.chdir(image_path)

    for current_dir, dirs, files in os.walk('.'):

      for f in files:

        if(f.endswith('.jpg')):

          image_title = f[:-4]

          YOLO_file = YOLO_VALUES.loc[YOLO_VALUES['ImageID'] == image_title]

          df = YOLO_file.loc[:,['classnumber', 'center x', 'center y', 'width', 'height']].copy()

          save_path = image_path + '/' + image_title + '.txt'

          df.to_csv(save_path, header=False, index = False, sep = ' ')

    def 


###Train & Test Path

In [66]:
class train_test:

  def path_list_train_test():

    import os
    image_path = '/content/drive/MyDrive/YOLOV4/darknet/data/Person_Car'
    os.chdir(image_path)
    
    path_list = []
    
    for current_dir, dirs, files in os.walk('.'):
      for f in files:

        if f.endswith('.jpg'):

          file_loc = image_path + '/' + f

          path_list.append(file_loc + '\n')

    #Train_test_Path 
    path_list_test = path_list[:int(len(path_list) * 0.20)]
    path_list = path_list[int(len(path_list) * 0.20):]

    #Created Train & Test Path File 
    with open('./train.txt', 'w') as train:
      for i in path_list:
        train.write(i)
        
    with open('./test.txt', 'w') as test:
      for i in path_list_test:
        test.write(i)

### Train on COCO DataSet

In [ ]:
class traincoco:

  def train_coco():
    
    import os
    path = '/content/drive/MyDrive/YOLOV4'
    os.chdir(path)

    #cloning darknet
    !git clone https://github.com/AlexeyAB/darknet

    #compile 
    os.chdir('/content/drive/MyDrive/YOLOV4/darknet')
    !make

    #Downloaded weights trained on COCO Dataset
    !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

    #test on a image. 
    !./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/image_000000015.jpg
  
  def print_image():
    
    import cv2 
    import matplotlib.pyplot as plt 
    %matplotlib inline

    image = cv2.imread('predictions.jpg')
    fig= plt.gcf()

    fig.set_size_inches(12,14)
    plt.imshow(image)

### Train & Test on Custom Dataset

In [ ]:
class customdataset:

  def training():

    #compile 
    import os
    os.chdir('/content/drive/MyDrive/YOLOV4/darknet')
    !sudo chmod +x darknet
    !./darknet

    # Train on Custom Data Person_Car
    !./darknet detector train data/Person_Car/image_data.data cfg/yolov4_train.cfg yolov4.conv.137.1 -dont_show

  def testing():

    #Testing on trained weights
    !./darknet detector train data/Person_Car/image_data.data cfg/yolov4_train.cfg /content/drive/MyDrive/YOLOV4/darknet/backup/yolov4_train_last.weights 
